In [1]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: ignored

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.preprocessing import StandardScaler , LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score , recall_score , precision_score , f1_score , ConfusionMatrixDisplay

In [ ]:
def print_metrics(y , pred_y , title = None):
    print(title)
    print(f"정확도 : {accuracy_score(y , pred_y)}")
    print(f"재현율 : {recall_score(y , pred_y , average = 'macro')}")
    print(f"정밀도 : {precision_score(y , pred_y , average = 'macro')}")
    print(f"f1 스코어 : {f1_score(y , pred_y , average = 'macro')}")

In [ ]:
path = '/content/drive/MyDrive/dacon/2023.5.18/open/'

In [ ]:
df = pd.read_csv(path + "train.csv")
df_test = pd.read_csv(path + "test.csv")

In [ ]:
check = df.drop(['ID' , '요일' , '범죄발생지'] , axis = 1)

vif2 = pd.DataFrame()
vif2['VIF'] = [variance_inflation_factor(check.values , i) for i in range(check.shape[1])]
vif2['features'] = check.columns
vif2 = vif2.round(2)
vif2 = vif2.sort_values(by = "VIF")

vif2

,VIF,features
10,1.23,짙은안개
15,1.45,눈날림
5,1.47,강수량(mm)
11,1.54,번개
14,1.59,연기/연무
12,1.60,진눈깨비
13,1.65,서리
6,1.76,강설량(mm)
7,1.77,적설량(cm)
16,1.95,TARGET


In [ ]:
# 소관지역으로 결정
df = df.drop(['ID' , '소관경찰서'] , axis = 1)
df_test = df_test.drop(['ID' , '소관경찰서'] , axis = 1)

In [ ]:
ordinal_features = ['요일', '범죄발생지']

for feature in ordinal_features:
    le = LabelEncoder()
    le = le.fit(df[feature])
    df[feature] = le.transform(df[feature])
    
    for label in np.unique(df[feature]):
        if label not in le.classes_:
            le.classes_ = np.append(le.classes_, label)
    df[feature] = le.transform(df[feature])

In [ ]:
ordinal_features = ['요일', '범죄발생지']

for feature in ordinal_features:
    le = LabelEncoder()
    le = le.fit(df_test[feature])
    df_test[feature] = le.transform(df_test[feature])

    # x_train데이터에서 존재하지 않았던 값이 x_test 데이터에 존재할 수도 있습니다.
    # 따라서 x_test 데이터를 바로 변형시키지 않고 고윳값을 확인후 x_test 데이터를 변환합니다.
    for label in np.unique(df_test[feature]):
        if label not in le.classes_:
            le.classes_ = np.append(le.classes_, label)
    df_test[feature] = le.transform(df_test[feature])

In [ ]:
train_y = df['TARGET'] # y값 
df = df.drop(['TARGET'] , axis = 1)

In [ ]:
# 스케일링

scale = StandardScaler()
scale_df = scale.fit_transform(df)
scale_df_test = scale.fit_transform(df_test)

scale_df

array([[ 0.82676039,  0.90582138, -1.3612488 , ..., -0.24650222,
        -0.09327388, -0.08397168],
       [ 1.4702024 , -0.21576762, -1.00078155, ..., -0.24650222,
        -0.09327388, -0.08397168],
       [ 0.50503939, -0.21576762,  1.45039575, ..., -0.24650222,
        -0.09327388, -0.08397168],
       ...,
       [ 0.18331839, -0.21576762,  1.3783023 , ..., -0.24650222,
        -0.09327388, -0.08397168],
       [-1.74700764,  1.18621863, -0.13566015, ..., -0.24650222,
        -0.09327388, -0.08397168],
       [ 0.18331839,  1.18621863, -1.00078155, ..., -0.24650222,
        -0.09327388, -0.08397168]])

In [ ]:
scale_df = pd.DataFrame(scale_df , columns = df.columns)

,월,시간,소관지역,사건발생거리,강수량(mm),강설량(mm),적설량(cm),풍향,안개,짙은안개,...,약국,은행,인도,주거지,주유소,주차장,차도,편의점,학교,호텔/모텔
0,0.826760,0.905821,-1.361249,0.728912,-0.392417,-0.144101,-0.275008,0.590789,-0.791919,-0.134783,...,-0.088299,-0.039577,-0.28733,-0.863995,-0.126238,-0.2005,1.50385,-0.246502,-0.093274,-0.083972
1,1.470202,-0.215768,-1.000782,1.352739,-0.392417,-0.144101,-0.275008,0.133001,-0.791919,-0.134783,...,-0.088299,-0.039577,-0.28733,-0.863995,-0.126238,-0.2005,1.50385,-0.246502,-0.093274,-0.083972
2,0.505039,-0.215768,1.450396,-0.305489,-0.392417,-0.144101,-0.275008,-1.494687,1.262756,-0.134783,...,-0.088299,-0.039577,3.48032,-0.863995,-0.126238,-0.2005,-0.66496,-0.246502,-0.093274,-0.083972
3,-0.460124,-0.215768,1.882956,0.009588,-0.211029,-0.144101,-0.275008,0.387328,1.262756,7.419332,...,-0.088299,-0.039577,-0.28733,1.157414,-0.126238,-0.2005,-0.66496,-0.246502,-0.093274,-0.083972
4,0.826760,1.186219,1.017835,-0.128009,-0.392417,-0.144101,-0.275008,0.692519,-0.791919,-0.134783,...,-0.088299,-0.039577,-0.28733,-0.863995,7.921537,-0.2005,-0.66496,-0.246502,-0.093274,-0.083972
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84401,-0.781845,0.064630,-1.144968,1.977742,1.188030,-0.144101,-0.275008,-0.223055,1.262756,-0.134783,...,-0.088299,-0.039577,-0.28733,-0.863995,-0.126238,-0.2005,1.50385,-0.246502,-0.093274,-0.083972
84402,0.505039,1.466616,0.801555,-0.473563,-0.392417,-0.144101,-0.275008,0.133001,-0.791919,-0.134783,...,-0.088299,-0.039577,-0.28733,-0.863995,-0.126238,-0.2005,1.50385,-0.246502,-0.093274,-0.083972
84403,0.183318,-0.215768,1.378302,1.077135,1.296109,-0.144101,-0.275008,1.302903,-0.791919,-0.134783,...,-0.088299,-0.039577,-0.28733,-0.863995,-0.126238,-0.2005,1.50385,-0.246502,-0.093274,-0.083972
84404,-1.747008,1.186219,-0.135660,-1.480094,-0.355968,0.398390,-0.148833,1.455498,1.262756,7.419332,...,-0.088299,-0.039577,-0.28733,1.157414,-0.126238,-0.2005,-0.66496,-0.246502,-0.093274,-0.083972


In [ ]:
scale_df_test = pd.DataFrame(scale_df_test , columns = df_test.columns)

In [ ]:
# 데이터 나누기

trainX , validX , trainY , validY = train_test_split(df , train_y , test_size = 0.2 , shuffle = True)

In [ ]:
# 모델

model = RandomForestClassifier()
model.fit(trainX , trainY) # 학습

RandomForestClassifier()

In [ ]:
prediction = model.predict(validX)

In [ ]:
print_metrics(validY.values , prediction)

None
정확도 : 0.5069304584764839
재현율 : 0.4770754234373349
정밀도 : 0.48949470203760753
f1 스코어 : 0.4786188099170325


# test.csv 예측

In [ ]:
submission = pd.read_csv("/content/drive/MyDrive/dacon/2023.5.18/open/sample_submission.csv")

In [ ]:
test_prediction = model.predict(df_test)

In [ ]:
submission['TARGET'] = test_prediction
submission['TARGET'].value_counts()

0    9160
1    4469
2    3660
Name: TARGET, dtype: int64

In [ ]:
submission.to_csv(path + "submit.csv" , index = False)